<a href="https://colab.research.google.com/github/kargaranamir/cluster-KenLM-PP/blob/main/cluster_kenlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [ ]:
!git clone https://github.com/kpu/kenlm.git

Cloning into 'kenlm'...
remote: Enumerating objects: 14142, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 14142 (delta 149), reused 394 (delta 124), pack-reused 13687
Receiving objects: 100% (14142/14142), 5.91 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (8029/8029), done.


In [ ]:
! mv kenlm kenlmgit

In [ ]:
%cd /content/kenlmgit
!mkdir -p build

/content/kenlmgit


In [ ]:
%cd /content/kenlmgit/build
!cmake ..

/content/kenlmgit/build
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found BZip2: /usr/lib/x86_64-

In [ ]:
!make -j 8

[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum.cc.o
[  3%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fast-dtoa.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fixed-dtoa.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/strtod.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/string-to-double.cc.o
[ 10%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-to-string.cc.o
[ 11%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/chain.cc.o
[ 12%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/count_records.cc.o
[ 13%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/io.cc.o
[ 15%] Building CXX obje

In [ ]:
%cd /content/kenlmgit
! pip install .
%cd /content

/content/kenlmgit
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/kenlmgit
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.0.0-cp39-cp39-linux_x86_64.whl size=3262389 sha256=caa7eff9e169c788964bc02182067fa9aef52dccf678c1d659a9af7d47fb782f
  Stored in directory: /tmp/pip-ephem-wheel-cache-874w_cu8/wheels/ba/1a/d9/1e191f320910c64fcc8e713de08c5132cbf9f549ec2a91f5b2
Successfully built kenlm
/content


In [ ]:
# Create file
!echo -e "hello world everyone\ngood afternoon ladies and gentleman\nmy friend in college\nhave a nice day\nspy and family at the same time\nwe do not do that here " >> example.txt

In [ ]:
# you don't need --discount_fallback if you use your real data (which may huge)
!kenlmgit/build/bin/lmplz -o 2 -S 80% -T /tmp <example.txt >example.arpa --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/example.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 29 types 30
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:360 2:10893066240
Substituting fallback discounts for order 0: D1=0.5 D2=1 D3+=1.5
Substituting fallback discounts for order 1: D1=0.5 D2=1 D3+=1.5
Statistics:
1 30 D1=0.5 D2=1 D3+=1.5
2 35 D1=0.5 D2=1 D3+=1.5
Memory estimate for binary LM:
type       B
probing 1420 assuming -p 1.5
probing 1544 assuming -r models -p 1.5
trie     930 without quantization
trie    1859 assuming -q 8 -b 8 quantization 
trie     930 assuming -a 22 array pointer compression
trie    1859 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:360 2:560
----5---10---1

In [ ]:
! rm example.arpa
! rm example.txt

## Main

In [ ]:
import os
import random
import time
import kenlm
from glob import glob
from tqdm import tqdm
import copy
import collections, functools, operator
import pandas as pd
from pathlib import Path
import re

In [ ]:
class Langcluster:
    def __init__(self, ngram_param, num_models, lmplz_path, dataset_path, models_path, split_data_path):
        self.ngram_param = int(ngram_param)
        self.num_models = int(num_models)
        self.lmplz_path = lmplz_path.rstrip('/')
        self.dataset_path = dataset_path.rstrip('/')
        self.models_path = models_path.rstrip('/')
        self.split_data_path = split_data_path.rstrip('/')
        self.target_data = []
        self.target_data_dict = {}
        self.size_data = 0
        self.models_dict = {} # {0: {indices: [], class: }, 1: [], 2: [], ....}
        self.BIG_PREP = 1000
        self.MAX_ITERATION = 1000_000
        self.NUM_SENTENCE_FIRST = 10
        self.NUM_SENTENCE_MORE = 10


        # create paths
        Path(models_path).mkdir(parents=True, exist_ok=True)
        Path(split_data_path).mkdir(parents=True, exist_ok=True)


    def read_main_file(self, header=True):
        # read target data
        self.target_data = []
        with open(self.dataset_path, 'r') as input_data:
            for index, line in enumerate(input_data):
                if index!=0 or (index==0 and header==False):
                    self.target_data.append(line.strip())
                else:
                    pass

        # update size of target data
        self.size_data = len(self.target_data)
        # convert data to dict
        self.convert_data_list_to_dict()

        return True

    def select_random_sent_seeds(self):
        selcted_seeds = random.sample(range(0, self.size_data), self.num_models)
        res_dict = {}
        for i in range(0, self.num_models):
            res_dict[i] = {}
            res_dict[i]['indices'] = []
            res_dict[i]['indices'].append(selcted_seeds[i])
            res_dict[i]['class'] = self.target_data[selcted_seeds[i]].strip().split('|')[0].strip()
            res_dict[i]['class_indices'] = [res_dict[i]['class']]
            res_dict[i]['index_plus_class'] = [f"{selcted_seeds[i]}: {res_dict[i]['class']}"]

        return res_dict

    def save_splitted_data(self):

        for key in self.models_dict.keys():
            data_indices_model_i = self.models_dict[key]['indices']
            data_model_i = []

            for sent_index in data_indices_model_i:
                sent = '|'.join(self.target_data[sent_index].strip().split('|')[1:]).strip()
                data_model_i.append(sent)

            with open(f"{self.split_data_path}/data_model_{key}", 'w') as f:
                f.write("\n".join(data_model_i))
                f.write("\n")

        return True

    def train_splitted_data_models(self):
        for key in tqdm(self.models_dict.keys()):

            command = ' '.join([f"{self.lmplz_path}", f"-o {str(self.ngram_param)}", f"<{self.split_data_path}/data_model_{key}", f">{self.models_path}/model_{key}.arpa", "--discount_fallback"])
            # print(f"\n **Model_NO_{key} train started**")
            os.system(command)
            # print(f"\n **Model_NO_{key} train finished**")

        return True


    def convert_data_list_to_dict(self):
        self.target_data_dict = {}

        for index, sent in enumerate(self.target_data):
            self.target_data_dict[index] = {}
            self.target_data_dict[index]['_sent'] = '|'.join(self.target_data[index].strip().split('|')[1:]).strip()
            self.target_data_dict[index]['_class'] = self.target_data[index].strip().split('|')[0].strip()

        return True


    @staticmethod
    def pp(log_score, length):
        return 10.0 ** (-log_score / length)


    def prepchar(self, model, model_lang, docs_dict_char):

        for doc_key in docs_dict_char.keys():
            try:
                line = docs_dict_char[doc_key]['_sent']
                log_score = model.score(line, bos=True, eos=True)
                length = len(line.split()) + 1

                prep_score = round(self.pp(log_score, length), 2)
                docs_dict_char[doc_key][model_lang] = prep_score
            except:
                docs_dict_char[doc_key][model_lang] = self.BIG_PREP

        return docs_dict_char


    def evaluate_models(self):
        input_model_paths = glob(f"{self.models_path}/*")
        for m_path in tqdm(input_model_paths):
            # open model
            model = kenlm.Model(m_path)
            self.target_data_dict = self.prepchar(model, m_path, self.target_data_dict)

        return True


    def compute_confidence(self):
        # compute confidence for each model and sentence
        input_model_paths = glob(f"{self.models_path}/*")
        for key in self.target_data_dict.keys():
            perp_models = [float(self.target_data_dict[int(key)][str(m_path)]) for m_path in input_model_paths]
            perp_models_sorted = sorted(perp_models)
            perfromance_key = perp_models_sorted[0] / (perp_models_sorted[1] - perp_models_sorted[0] + + 0.0001)
            min_model_index = int(perp_models.index(perp_models_sorted[0]))

            self.target_data_dict[key]['_min_model'] = str(input_model_paths[min_model_index])
            self.target_data_dict[key]['_min_model_performance'] = float(perfromance_key)

        return True

    def select_next_candidates(self, num_top_sent=10):

        input_dict = self.models_dict.copy()
        total_selected_indices = []

        for key in input_dict.keys():
            total_selected_indices = total_selected_indices + input_dict[key]['indices']

        sorted_choices = list(sorted(self.target_data_dict, key=lambda x:self.target_data_dict[x]['_min_model_performance'], reverse=False))

        # remove selected ones:
        for num in total_selected_indices:
            sorted_choices.remove(num)


        candidates_top = sorted_choices[:num_top_sent]
        for num in candidates_top:
            model_no = int(self.target_data_dict[num]['_min_model'].split('/')[-1].replace('.arpa', '').split('_')[-1])
            sent_script_class = self.target_data_dict[num]['_class']
            input_dict[model_no]['indices'].append(num)
            input_dict[model_no]['class_indices'].append(sent_script_class)
            input_dict[model_no]['index_plus_class'].append(f"{num}: {sent_script_class}")


        return input_dict


    def first_iteration(self):
        self.read_main_file(header=True)
        self.models_dict = self.select_random_sent_seeds()
        self.save_splitted_data()
        self.train_splitted_data_models()
        self.evaluate_models()
        self.compute_confidence()
        self.models_dict = self.select_next_candidates(num_top_sent=self.NUM_SENTENCE_FIRST)


    def more_iteration(self, num_iteration=1000_000):
        for _ in tqdm(range(0, num_iteration)):

            prev_models_dict = copy.deepcopy(self.models_dict)

            # save selected data
            self.save_splitted_data()
            # train models
            self.train_splitted_data_models()
            # evaluate models
            self.evaluate_models()
            # compute confidence
            self.compute_confidence()
            # assign next sentences
            self.models_dict = self.select_next_candidates(num_top_sent=self.NUM_SENTENCE_MORE)

            if self.models_dict == prev_models_dict:
                return True


    def train(self):
        self.first_iteration()
        self.more_iteration(num_iteration=self.MAX_ITERATION)

    def predict(self):
        input_model_paths = glob(f"{self.models_path}/*")
        model_paths_no = [int(key.split('/')[-1].replace('.arpa', '').split('_')[-1]) for key in input_model_paths]

        eval_model = {}
        for model_no in model_paths_no:
            model_i_indices = self.models_dict[model_no]['indices']

            # get dicts of sentences as a list
            sent_dicts_i = [self.target_data_dict[sent_i] for sent_i in model_i_indices]
            # only the models keys
            sent_dicts_i_restricted = [{key: item[key] for key in input_model_paths} for item in sent_dicts_i]
            # sum over list of dicts
            eval_result_i = dict(functools.reduce(operator.add, map(collections.Counter, sent_dicts_i_restricted)))
            # mean
            eval_result_i_mean = {k: v / len(model_i_indices) for k, v in eval_result_i.items()}

            # rename keys
            for o_key, n_key in zip(input_model_paths, model_paths_no):
                eval_result_i_mean[n_key] = eval_result_i_mean.pop(o_key)

            # save
            eval_model[model_no] = eval_result_i_mean # {0: {0:8, 1:9, ....}}

        return eval_model


In [ ]:
# convert setntence to characters
def tokenizer_char(sentence: str) -> str:
    if sentence == "":
        return ""
    elif sentence.strip() == "":
        return "<sp>"
    else:
        s = ' '.join(list(sentence))
        # replace space with <space> token
        s = re.sub(r"\s\s+", ' <sp> ', s)
        return s.strip()

In [ ]:
def char_to_word(line):
    line_list = line.split()
    line_list = [x if x!='<sp>' else ' ' for x in  line_list]
    return ''.join(line_list)

In [ ]:
! mkdir data
! rm -r split_data
! rm -r models
! mkdir split_data
! mkdir models

rm: cannot remove 'split_data': No such file or directory
rm: cannot remove 'models': No such file or directory


In [ ]:
# data_df = pd.read_csv('/content/data/tgk_Arab.csv')
# data_df['_sent_char'] = data_df['_sent'].apply(lambda x: tokenizer_char(str(x)))
# new_data_df = data_df[['0_source_lang', '_sent_char']][:100]
# new_data_df.to_csv('/content/data/tgk_Arab', sep='|', index=False)

In [ ]:
# tgk_obj = Langcluster(ngram_param=3,
#                       num_models=10,
#                       lmplz_path = "/content/kenlmgit/build/bin/lmplz",
#                       dataset_path = "/content/data/tgk_Arab",
#                       models_path = "/content/models/tgk_Arab",
#                       split_data_path = "/content/split_data/tgk_Arab")

In [ ]:
# tgk_obj.train()

In [ ]:
# # predict 10X10 and save
# res_predict = tgk_obj.predict()
# df_res_predict = pd.DataFrame(res_predict)
# df_res_predict = df_res_predict.sort_index(axis=1)
# df_res_predict = df_res_predict.sort_index(axis=0)
# df_res_predict.to_csv(f'/content/split_data/tgk_Arab/log.csv')
# ! zip -r split_data_tgk_Arab.zip /content/split_data/tgk_Arab
# files.download('split_data_tgk_Arab.zip')

## Practice

In [ ]:
def practice(language_name):

    # prepare data
    data_df = pd.read_csv(f'/content/data/{language_name}.csv')
    data_df['_sent_char'] = data_df['_sent'].apply(lambda x: tokenizer_char(str(x)))
    new_data_df = data_df[['0_source_lang', '_sent_char']][:1000]
    new_data_df.to_csv(f'/content/data/{language_name}', sep='|', index=False)


    # create object
    obj = Langcluster(ngram_param=3,
                          num_models=10,
                          lmplz_path = "/content/kenlmgit/build/bin/lmplz",
                          dataset_path = f"/content/data/{language_name}",
                          models_path = f"/content/models/{language_name}",
                          split_data_path = f"/content/split_data/{language_name}")

    # train
    obj.train()

    # save data of other models
    for model_no in obj.models_dict.keys():
        model_indices_i = obj.models_dict[model_no]['indices']
        data_df.loc[data_df.index.isin(model_indices_i)].to_csv(f'/content/split_data/{language_name}/data_model_{model_no}.csv')

    # predict 10X10 and save
    res_predict = obj.predict()
    df_res_predict = pd.DataFrame(res_predict)
    df_res_predict = df_res_predict.sort_index(axis=1)
    df_res_predict = df_res_predict.sort_index(axis=0)
    df_res_predict.to_csv(f'/content/split_data/{language_name}/log.csv')



In [ ]:
from google.colab import files

In [ ]:
practice('ach_Latn')
! zip -r split_data_ach_Latn.zip /content/split_data/ach_Latn
files.download('split_data_ach_Latn.zip')

In [ ]:
practice('luo_Latn')
! zip -r split_data_luo_Latn.zip /content/split_data/luo_Latn
files.download('split_data_luo_Latn.zip')

In [ ]:
practice('teo_Latn')
! zip -r split_data_teo_Latn.zip /content/split_data/teo_Latn
files.download('split_data_teo_Latn.zip')